In [1]:
# import packages
import numpy as np
import pandas as pd
import os
import seaborn as sns
import Bio
import matplotlib.pyplot as plt
import ast
import scipy.stats as stats
from scipy.optimize import curve_fit

In [3]:
# import data

path: str = './data/' #folder where files are stored
#prokaryotes_lysate: pd.DataFrame = pd.read_csv(os.path.join(path, "prokaryotes_322columns.csv"))
prokaryotes_all: pd.DataFrame = pd.read_csv(os.path.join(path, "prokaryotes_all.csv"), dtype=str)
prokaryotes_values: pd.DataFrame = pd.read_csv(os.path.join(path, "prokaryotes_funct.csv"), dtype=str)
prokaryotes_values['a_point'] = prokaryotes_values['a_point'].astype('float64')
prokaryotes_values['b_point'] = prokaryotes_values['b_point'].astype('float64')
prokaryotes_values['plateau'] = prokaryotes_values['plateau'].astype('float64')
prokaryotes_values['meltPoint'] = prokaryotes_values['meltPoint'].astype('float64')

In [ ]:
# x = np.array([1.1, 2.1, 3, 4.5, 5.3, 6])
# y = np.array([1, 2.4, 3.1, 4.4, 5, 6.7])
# plt.plot(x, y, 'o')

# def linear(x, A, B):
#     y = A*x + B
#     return y

# parameters, covariance = curve_fit(linear, x, y)

# fit_y = linear(x, parameters[0], parameters[1])
# plt.plot(x, y, 'o', label='data')
# plt.plot(x, fit_y, '-', label='fit')
# plt.legend()

In [ ]:
# protein = prokaryotes_all.groupby('Protein_ID')
# protein_keys = list(protein.groups.keys())
# temp = np.asarray(protein.get_group(protein_keys[0])['temperature'])
# fold_change = np.asarray((protein.get_group(protein_keys[0])['fold_change']).astype('float64').min()/protein.get_group(protein_keys[0])['fold_change'].astype('float64'))
# plt.plot(temp, fold_change, 'o')


In [5]:
def curve_equation(x, a, b, plateau, target):
    rel_amount = ((1-plateau)/(1+(np.power(np.e, (-a/x)+b)))) + plateau
    return rel_amount - target
from scipy.optimize import fsolve
prokaryotes_values['solution'] = pd.Series(prokaryotes_values.index).apply(lambda i: fsolve(curve_equation, prokaryotes_values.loc[i, 'meltPoint'], args=(prokaryotes_values.loc[i,'a_point'], prokaryotes_values.loc[i,'b_point'], prokaryotes_values.loc[i,'plateau'], 0.1)))
essential_proteins_bs = prokaryotes_values[(prokaryotes_values['Sample'] == 'B.subtilis_P023755') & (prokaryotes_values['solution'] < 54) & (prokaryotes_values['solution'] > 52)]


/var/folders/pm/vzcy9py91xzg_z5d80js6mtc0000gn/T/ipykernel_85560/258648289.py:5: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  prokaryotes_values['solution'] = pd.Series(prokaryotes_values.index).apply(lambda i: fsolve(curve_equation, prokaryotes_values.loc[i, 'meltPoint'], args=(prokaryotes_values.loc[i,'a_point'], prokaryotes_values.loc[i,'b_point'], prokaryotes_values.loc[i,'plateau'], 0.1)))
